## Setting Up

### Installing Dependencies

In [1]:
%pip install openai pydantic instructor python-dotenv
from IPython.display import clear_output ; clear_output()

### Loading Azure Open AI configuration

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")

from openai import AzureOpenAI

aoai = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

## Using Structured Input/Output with OpenAI

In [34]:
import instructor
from pydantic import BaseModel, Field
from typing import List
from enum import Enum
import json

client = instructor.from_openai(aoai)

def print_json(data):
    print(json.dumps(data, indent=2))

def print_schema(model: BaseModel):
    print_json(model.model_json_schema(), indent=2)

def print_result(result: BaseModel):
    print(result.model_dump_json(indent=2))

def llm(response_model: BaseModel = BaseModel, system: str = None, user: str = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    if user:
        messages.append({"role": "user", "content": user})
    result = client.chat.completions.create(
        model=GPT_4_O_MODEL_NAME,
        response_model=response_model,
        messages=messages,
    )
    return result

In [26]:
class AddressInfo(BaseModel):
    first_name: str
    last_name: str
    street: str
    house_number: str
    postal_code: str
    city: str
    state: str
    country: str

print_schema(AddressInfo)

{
  "properties": {
    "first_name": {
      "title": "First Name",
      "type": "string"
    },
    "last_name": {
      "title": "Last Name",
      "type": "string"
    },
    "street": {
      "title": "Street",
      "type": "string"
    },
    "house_number": {
      "title": "House Number",
      "type": "string"
    },
    "postal_code": {
      "title": "Postal Code",
      "type": "string"
    },
    "city": {
      "title": "City",
      "type": "string"
    },
    "state": {
      "title": "State",
      "type": "string"
    },
    "country": {
      "title": "Country",
      "type": "string"
    }
  },
  "required": [
    "first_name",
    "last_name",
    "street",
    "house_number",
    "postal_code",
    "city",
    "state",
    "country"
  ],
  "title": "AddressInfo",
  "type": "object"
}


In [35]:

address_info = llm(
    AddressInfo,
    "Eleanor Berger lives in Switzerland, in the canton of Zürich. Her home is at Freilagerstrasse 97, 8047 Zürich."
)

print_result(address_info)

{
  "first_name": "Eleanor",
  "last_name": "Berger",
  "street": "Freilagerstrasse",
  "house_number": "97",
  "postal_code": "8047",
  "city": "Zürich",
  "state": "Zürich",
  "country": "Switzerland"
}


## Working with Text

### Translation and Normalization

In [12]:
class TranslatedString(BaseModel):
    input_language: str = Field(..., description="The language of the original text, as 2-letter language code.")
    translation: str

translation = llm(
    TranslatedString,
    "Detect the language of the original text and translate it into English.",
    "Sprachkenntnisse sind ein wichtiger Bestandteil der Kommunikation.",
)

print(translation)

input_language='de' translation='Language skills are an important part of communication.'


In [36]:
class Translation(BaseModel):
    language: str = Field(..., description="The language of the translated text, as 2-letter language code.")
    translation: str

class Translations(BaseModel):
    translations: List[Translation]

target_languages = ["de", "fr", "it", "es", "he"]

translations = llm(
    Translations,
    f"Translate the user-provided text into the following languages: {json.dumps(target_languages)}",
    "Large Language Models are a powerful tool for natural language processing.",
)

print_result(translations)

{
  "translations": [
    {
      "language": "de",
      "translation": "Große Sprachmodelle sind ein mächtiges Werkzeug für die Verarbeitung natürlicher Sprache."
    },
    {
      "language": "fr",
      "translation": "Les grands modèles de langage sont un outil puissant pour le traitement du langage naturel."
    },
    {
      "language": "it",
      "translation": "I modelli linguistici di grandi dimensioni sono uno strumento potente per l'elaborazione del linguaggio naturale."
    },
    {
      "language": "es",
      "translation": "Los grandes modelos de lenguaje son una herramienta poderosa para el procesamiento del lenguaje natural."
    },
    {
      "language": "he",
      "translation": "מודלים של שפה גדולים הם כלי עוצמתי לעיבוד שפה טבעית."
    }
  ]
}


In [37]:

text_styles = ["formal", "informal", "casual", "academic", "professional", "business"]

class NormalizedText(BaseModel):
    style: str = Field(
        ...,
        description=(f"The style of the text normalization.  Options: {json.dumps(text_styles)}"))
    text: str

class NormalizedTexts(BaseModel):
    normalized_texts: List[NormalizedText]

normalizations = llm(
    NormalizedTexts,
    f"Normalize the user-provided text into the following styles: {json.dumps(text_styles)}",
    "Large Language Models are a powerful tool for natural language processing.",
)

print_result(normalizations)

{
  "normalized_texts": [
    {
      "style": "formal",
      "text": "Large Language Models represent a significant advancement in the domain of natural language processing."
    },
    {
      "style": "informal",
      "text": "Large Language Models are a really cool tool for getting computers to understand language."
    },
    {
      "style": "casual",
      "text": "Those Large Language Models are pretty awesome for handling language processing."
    },
    {
      "style": "academic",
      "text": "Large Language Models constitute a substantial development within the field of natural language processing, offering advanced capabilities in understanding and generating human language."
    },
    {
      "style": "professional",
      "text": "Large Language Models offer robust solutions for natural language processing tasks, enhancing computational understanding and generation of human language."
    },
    {
      "style": "business",
      "text": "Leveraging Large Language M

## Decision Making

## Planning and Tool-Use